## Инициализайия и нормализация

В этом задании вам предстоит реализовать два вида нормализации: по батчам (BatchNorm1d) и по признакам (LayerNorm1d).

In [6]:
from typing import Callable, NamedTuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

### 1. Реализация BatchNorm1d и LayerNorm1d.

    #### 1.1. (2 балла) Реализуйте BatchNorm1d

Подсказка: чтобы хранить текущие значения среднего и дисперсии, вам потребуется метод `torch.nn.Module.register_buffer`, ознакомьтесь с документацией к нему. Подумайте, какие проблемы возникнут, если вы будете просто сохранять ваши значения в тензор

In [7]:
class BatchNorm1d(nn.Module):
    def __init__(self, num_features: int, momentum: float = 0.9, eps: float = 1e-5) -> None:
        super().__init__()
        self.scale = nn.Parameter(torch.ones(num_features))
        self.shift = nn.Parameter(torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))
        self.momentum = momentum
        self.eps = eps

    def forward(self, x: Tensor) -> Tensor:
        if self.training:
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0)

            self.running_mean = (1 - self.momentum) * batch_mean + self.momentum * self.running_mean
            self.running_var = (1 - self.momentum) * batch_var + self.momentum * self.running_var

            x_normalized = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
        else:
            x_normalized = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)

        return self.scale * x_normalized + self.shift

#### 1.2. (1 балл) Реализуйте LayerNorm1d

Отличия LayerNorm от BatchNorm - в том, что расчёт средних и дисперсий в BatchNorm происходит вдоль размерности батча (см. рисунок слева), а в LayerNorm - вдоль размерности признаков (см. рисунок справа).

<img src="norm.png" width="800">

In [8]:
class LayerNorm1d(nn.Module):
    def __init__(self, num_features: int, eps: float = 1e-5) -> None:
        super(LayerNorm1d, self).__init__()
        self.scale = nn.Parameter(torch.ones(num_features))
        self.shift = nn.Parameter(torch.zeros(num_features))
        self.eps = eps

    def forward(self, x: Tensor) -> Tensor:
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True)
        
        x_normalized = (x - mean) / torch.sqrt(var + self.eps)
        
        return self.scale * x_normalized + self.shift

### 2. Эксперименты

В этом задании ваша задача - проверить, какие из приёмов хорошо справляются с нездоровыми активациями в промежуточных слоях. Вам будет дана базовая модель, у которой есть проблемы с инициализацией параметров, попробуйте несколько приёмов для устранения проблем обучения:
1. Хорошая инициализация параметров
2. Ненасыщаемая функция активации (например, `F.leaky_relu`)
3. Нормализация по батчам или по признакам (можно использовать встроенные `nn.BatchNorm1d` и `nn.LayerNorm`)
4. Более продвинутый оптимизатор (`torch.optim.RMSprop`)

#### 2.0. Подготовка: датасет, функции для обучения

Проверять наши гипотезы будем на датасете MNIST, для отладки добавим в функции для обучения возможность использовать только несколько батчей данных

In [9]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_dataset = datasets.MNIST(
    "data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
test_dataset = datasets.MNIST(
    "data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
def training_step(
    batch: tuple[torch.Tensor, torch.Tensor],
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
) -> torch.Tensor:
    # прогоняем батч через модель
    x, y = batch
    logits = model(x)
    # оцениваем значение ошибки
    loss = F.cross_entropy(logits, y)
    # обновляем параметры
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # возвращаем значение функции ошибки для логирования
    return loss


def train_epoch(
    dataloader: DataLoader,
    model: nn.Module,
    optimizer: torch.optim.Optimizer,
    max_batches: int = 100,
) -> Tensor:
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        loss = training_step(batch, model, optimizer)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()


@torch.no_grad()
def test_epoch(
    dataloader: DataLoader, model: nn.Module, max_batches: int = 100
) -> Tensor:
    loss_values: list[float] = []
    for i, batch in enumerate(dataloader):
        x, y = batch
        logits = model(x)
        # оцениваем значение ошибки
        loss = F.cross_entropy(logits, y)
        loss_values.append(loss.item())
        if i == max_batches:
            break
    return torch.tensor(loss_values).mean()

#### 2.1. Определение класса модели (2 балла)

Для удобства проведения экспериментов мы немного усложним создание модели, чтобы можно было задать разные способы инициализации параметров и нормализации промежуточных активаций, не меняя определение класса.

Добавьте в метод `__init__`:
- аргумент, который позволит использовать разные функции активации для промежуточных слоёв
- аргумент, который позволит задавать разные способы нормализации: `None` (без нормализации), `nn.BatchNorm` и `nn.LayerNorm`

In [11]:
def init_std_normal(model: nn.Module) -> None:
    """Функция для инициализации параметров модели стандартным нормальным распределением."""
    for param in model.parameters():
        torch.nn.init.normal_(param.data, mean=0, std=1)


from typing import Type


class MLP(nn.Module):
    """Базовая модель для экспериментов

    Args:
        input_dim (int): размерность входных признаков
        hidden_dim (int): размерност скрытого слоя
        output_dim (int): кол-во классов
        act_fn (Callable[[Tensor], Tensor], optional): Функция активации. Defaults to F.tanh.
        init_fn (Callable[[nn.Module], None], optional): Функция для инициализации. Defaults to init_std_normal.
        norm (Type[nn.BatchNorm1d  |  nn.LayerNorm] | None, optional): Способ нормализации промежуточных активаций.
            Defaults to None.
    """
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        output_dim: int,
        act_fn: Callable[[Tensor], Tensor] = F.tanh,
        init_fn: Callable[[nn.Module], None] = init_std_normal,
        norm: Type[nn.BatchNorm1d | nn.LayerNorm] | None = None,
    ) -> None:
        super().__init__()
        # теперь линейные слои будем задавать
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.act_fn = act_fn
        self.norm = norm(hidden_dim) if norm else None

        # reinitialize parameters
        init_fn(self)

    def forward(self, x: Tensor) -> Tensor:
        h = self.fc1.forward(x.flatten(1))
        # here you can do normalization
        if self.norm:
            h = self.norm(h)
        return self.fc2.forward(self.act_fn(h))

#### 2.2. Эксперименты (7 баллов)

Проведите по 3 эксперимента с каждой из модификаций с разными значениями `seed`, соберите статистику значений тестовой ошибки после 10 эпох обучения, сделайте выводы о том, что работает лучше

Проверяем:
1. Метод инициализации весов модели: $\mathcal{N}(0, 1)$ / Kaiming normal
2. Функция активации: tanh /  (или любая другая без насыщения)
3. Слой нормализации: None / BatchNorm / LayerNorm
4. Выбранный оптимизатор: SGD / RMSprop / Adam

Итого у нас 2 + 2 + 3 + 3 = 10 экспериментов, каждый нужно повторить 3 раза, посчитать среднее и вывести результаты в pandas.DataFrame.
Можно дополнительно потестировать разные сочетания опций, например инициализация + нормализация


Чтобы автоматизировать проведение экспериментов, можно использовать функцию, которая будет принимать все необходимые настройки эксперимента, запускать его и сохранять нужные метрики:

In [12]:
def run_experiment(
    model_gen: Callable[[], nn.Module],
    optim_gen: Callable[[nn.Module], torch.optim.Optimizer],
    seed: int,
    n_epochs: int = 10,
    max_batches: int | None = None,
    verbose: bool = False,
) -> float:
    """Функция для запуска экспериментов.

    Args:
        model_gen (Callable[[], nn.Module]): Функция для создания модели
        optim_gen (Callable[[nn.Module], torch.optim.Optimizer]): Функция для создания оптимизатора для модели
        seed (int): random seed
        n_epochs (int, optional): Число эпох обучения. Defaults to 10.
        max_batches (int | None, optional): Если указано, только `max_batches` минибатчей
            будет использоваться при обучении и тестировании. Defaults to None.
        verbose (bool, optional): Выводить ли информацию для отладки. Defaults to False.

    Returns:
        float: Значение ошибки на тестовой выборке в конце обучения
    """
    torch.manual_seed(seed)
    # создадим модель и выведем значение ошибки после инициализации
    model = model_gen()
    optim = optim_gen(model)
    epoch_losses: list[float] = []
    for i in range(n_epochs):
        train_loss = train_epoch(train_loader, model, optim, max_batches=max_batches)
        test_loss = test_epoch(test_loader, model, max_batches=max_batches)
        if verbose:
            print(f"Epoch {i} train loss = {train_loss:.4f}")
            print(f"Epoch {i} test loss = {test_loss:.4f}")

        epoch_losses.append(test_loss.item())

    last_epoch_loss = epoch_losses[-1]
    return last_epoch_loss

Пример использования:

In [13]:
losses = run_experiment(
    model_gen=lambda: MLP(784, 128, 10, init_fn=init_std_normal, norm=None),
    optim_gen=lambda x: torch.optim.SGD(x.parameters(), lr=0.01),
    seed=42,
    n_epochs=10,
    max_batches=100,
    verbose=True,
)

Epoch 0 train loss = 12.6168
Epoch 0 test loss = 9.9327
Epoch 1 train loss = 9.0954
Epoch 1 test loss = 7.5498
Epoch 2 train loss = 6.9607
Epoch 2 test loss = 6.2342
Epoch 3 train loss = 5.8992
Epoch 3 test loss = 5.3655
Epoch 4 train loss = 4.9951
Epoch 4 test loss = 4.7433
Epoch 5 train loss = 4.4778
Epoch 5 test loss = 4.3001
Epoch 6 train loss = 3.9693
Epoch 6 test loss = 3.9605
Epoch 7 train loss = 3.7261
Epoch 7 test loss = 3.6844
Epoch 8 train loss = 3.4223
Epoch 8 test loss = 3.4538
Epoch 9 train loss = 2.9975
Epoch 9 test loss = 3.2638


Для удобства задания настроек эксперимента можно определять их с помощью класса `Experiment`, в котором можно также реализовать логику для строкового представления:

In [14]:
input_dim = 784
hidden_dim = 128
output_dim = len(train_dataset.classes)


class Experiment(NamedTuple):
    init_fn: Callable[[nn.Module], None]
    act_fn: Callable[[Tensor], Tensor]
    norm: Type[nn.BatchNorm1d | nn.LayerNorm] | None
    optim_cls: Type[torch.optim.Optimizer]

    @property
    def model_gen(self) -> Callable[[], nn.Module]:
        return lambda: MLP(
            input_dim, hidden_dim, output_dim, init_fn=self.init_fn, norm=self.norm
        )

    @property
    def optim_gen(self) -> Callable[[nn.Module], torch.optim.Optimizer]:
        return lambda x: self.optim_cls(x.parameters(), lr=0.01)

    def __repr__(self) -> str:
        norm = None if self.norm is None else self.norm.__name__
        exp_params = (f"Experiment parameters: "
                      f"{self.init_fn.__name__}, {self.act_fn.__name__}, {norm}, {self.optim_cls.__name__}")
        return exp_params

Описываем все эксперименты:

In [15]:
# Все описано дальше

'''
options = [
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.tanh,
        norm=None,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.silu,
        norm=nn.LayerNorm,
        optim_cls=torch.optim.SGD,
    ),
    Experiment(
        init_fn=init_std_normal,
        act_fn=F.relu,
        norm=nn.BatchNorm1d,
        optim_cls=torch.optim.RMSprop,
    ),
]
'''

'\noptions = [\n    Experiment(\n        init_fn=init_std_normal,\n        act_fn=F.tanh,\n        norm=None,\n        optim_cls=torch.optim.SGD,\n    ),\n    Experiment(\n        init_fn=init_std_normal,\n        act_fn=F.silu,\n        norm=nn.LayerNorm,\n        optim_cls=torch.optim.SGD,\n    ),\n    Experiment(\n        init_fn=init_std_normal,\n        act_fn=F.relu,\n        norm=nn.BatchNorm1d,\n        optim_cls=torch.optim.RMSprop,\n    ),\n]\n'

In [16]:
import itertools

def init_kaiming(model: nn.Module) -> None: 
    for param in model.parameters():
        if param.dim() > 1:
            nn.init.kaiming_normal_(param.data, mode='fan_in', nonlinearity='relu')
        else:
            nn.init.zeros_(param.data)


# Сбор различных параметров 
inits = [init_std_normal, init_kaiming]
acts = [F.tanh, F.silu, F.relu]
norms = [None, nn.BatchNorm1d, nn.LayerNorm]
optims = [torch.optim.SGD, torch.optim.RMSprop, torch.optim.Adam]

full_exp_options = list(itertools.product(inits, acts, norms, optims))

exp_options = []
for option in full_exp_options:
    exp_options.append(Experiment(*option))

Запускаем расчёты:

In [17]:
from joblib import Parallel, delayed
import itertools

seeds = [42, 56, 12]  # здесь вам нужно 3 разных значения
 
full_options = list(itertools.product(exp_options, seeds))

def get_result(option, seed):        
    loss = run_experiment(
        model_gen=lambda: option.model_gen(),
        optim_gen=lambda x: option.optim_gen(x),
        seed=seed,
        n_epochs=10,
        max_batches=None,
        verbose=True,
    )    
    return [str(option), seed, loss]

with Parallel(n_jobs=-2, verbose=10) as parallel:
    results = parallel(delayed(get_result)(option[0], option[1]) for option in full_options)
      

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed: 22.4min
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed: 26.9min
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed: 30.2min
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed: 36.2min
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed: 41.8min
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed: 46.9min
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed: 53.0min
[Parallel(n_jobs=-2)]: Done 162 out of 162 | elapsed: 56.7min finished


In [18]:
# Вариант кода без распараллеливания 

"""
seeds = [42, 56, 12]  # здесь вам нужно 3 разных значения
results = []

for option in options:
    print(option)
    for seed in seeds:
        loss = run_experiment(
            model_gen=lambda: option.model_gen(),
            # model_gen=option.model_gen(),
            optim_gen=lambda x: option.optim_gen(x),
            # optim_gen=option.optim_gen(option.model_gen()),
            seed=seed,
            n_epochs=10,
            max_batches=None,
            verbose=True,
        )
        results.append([str(option), seed, loss])   
"""

'\nseeds = [42, 56, 12]  # здесь вам нужно 3 разных значения\nresults = []\n\nfor option in options:\n    print(option)\n    for seed in seeds:\n        loss = run_experiment(\n            model_gen=lambda: option.model_gen(),\n            # model_gen=option.model_gen(),\n            optim_gen=lambda x: option.optim_gen(x),\n            # optim_gen=option.optim_gen(option.model_gen()),\n            seed=seed,\n            n_epochs=10,\n            max_batches=None,\n            verbose=True,\n        )\n        results.append([str(option), seed, loss])   \n'

    Выводим результаты:

In [19]:
import pandas as pd

# преобразование описания эксперимента для разнесения параметров на разные колонки
for i in range(len(results)):
    results[i] = results[i][0][23:].split(", ") + results[i][1:]

pd.DataFrame(results, 
             columns=["Initialization function", "Activation function", 
                      "Normalization function", "Optimization function", 
                      "Seed", "Loss"])

,Initialization function,Activation function,Normalization function,Optimization function,Seed,Loss
0,init_std_normal,tanh,None,SGD,42,0.634049
1,init_std_normal,tanh,None,SGD,56,0.643496
2,init_std_normal,tanh,None,SGD,12,0.693835
3,init_std_normal,tanh,None,RMSprop,42,0.177688
4,init_std_normal,tanh,None,RMSprop,56,0.260855
...,...,...,...,...,...,...
157,init_kaiming,relu,LayerNorm,RMSprop,56,0.109345
158,init_kaiming,relu,LayerNorm,RMSprop,12,0.131263
159,init_kaiming,relu,LayerNorm,Adam,42,0.137919
160,init_kaiming,relu,LayerNorm,Adam,56,0.121653


ВЫВОДЫ:

Из полученной таблицы явно видно, что все лучшие результаты (для любого сида) получены при использовании LayerNorm и RMSprop. При таком выборе нормализации и оптимизации лучшей функцией инициализации показала себя Kaimimg Normal, однако стандартное нормальное распределение (init_std_normal) при инициализации лишь незначительно хуже. Касательно функции активации вывод сделать невозможно, так как явных зависимостей в сравнении с другими параметрами не прослеживается.
